In [ ]:
import spacy
import pandas as pd
from collections import Counter

In [ ]:
# 1. 口コミデータの読み込み
df = pd.read_csv('text/usj.csv', encoding='utf-8',
                 parse_dates=['旅行の時期'])

# 口コミに含まれている空行を削除
df['口コミ'] = df['口コミ'].replace('\n+', '\n', regex=True)

# 旅行の時期が欠損している口コミを削除する
df = df.dropna(subset=['旅行の時期'])

# 旅行の時期を年度に直し，「年度」という項目を追加
df['年度'] = df['旅行の時期'].apply(lambda x: x.year if x.month >= 4 else x.year-1)

# 2. 2013年度までの口コミと2014年度からの口コミに分割
df2013 = df[(df['年度'] >= 2011) & (df['年度'] <= 2013)]
df2014 = df[(df['年度'] >= 2014) & (df['年度'] <= 2018)]

In [ ]:
nlp = spacy.load("ja_ginza")

# 使用する単語の品詞とストップワードの指定
include_pos = ('NOUN', 'PROPN')
stopwords = ('こと', 'なし', 'ため', 'よう', 'ところ',
             'もの', 'あと', 'さん', '以上', '以外')

In [ ]:
# 3. 単語の頻度をカウントする関数を用意
def count_words(df):
    # 出現する単語をリストに追加
    words = []
    for doc in nlp.pipe(df["口コミ"]):
        words.extend([token.lemma_ for token in doc
                      if token.pos_ in include_pos and
                      token.lemma_ not in stopwords])

    # 単語の頻度をカウント
    counter = Counter(words)
    return counter


# 4. 2013年度までの口コミと2014年度からの口コミの単語の頻度をカウント
counter2013 = count_words(df2013)   # 2013年度までの単語の頻度をカウント
counter2014 = count_words(df2014)   # 2014年度からの単語の頻度をカウント

In [ ]:
# 5. 単語の増加率を算出
word2rate = Counter()                   # 単語と増加率を保存する辞書
for word, count in counter2014.items():
    if len(word) > 1 and count >= 100:
        if word in counter2013:
            rate = count / counter2013[word]
            if rate >= 3.0:
                word2rate[word] = round(rate, 1)
        else:
            # 2013年度までの口コミに単語が出現しない場合は増加率を1000.0としておく
            word2rate[word] = 1000.0

# 6. 単語の増加率を表示
print('    rate word')
for word, rate in word2rate.most_common():
    print(f'{rate:>8} {word}')